# Import

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import bz2
import urllib
from datetime import datetime, date

data_folder = 'data/'

In [17]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/theopatron/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Creation of extended files

### FOR THE TIMELINE
keep quotation mentionning guns 

### FOR THE ANALYSIS
keep quotation mentionning guns <br/>
keep only those that have a speaker <br/>
keep american speakers <br/>
add colums corresponding to characteristics of speaker <br/>

In [18]:
lexical_field = ['guns', 'rifle', 'slaughter'] # To be continued

speakers = pd.read_parquet(data_folder + 'speaker_attributes.parquet')
label = pd.read_csv(data_folder + 'wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')

In [65]:
speakers.loc[123].date_of_birth[0]

TypeError: 'NoneType' object is not subscriptable

In [54]:
datetime(int('0050'), int('00'), int('00'))

ValueError: month must be in 1..12

In [73]:
try:
    born = datetime.strptime(speakers.loc[125].date_of_birth[0][1:11], "%Y-%m-%d").date()
    today = date.today()
    age = today.year - born.year - ((today.month, today.day) < (born.month, born.day))
except:
    age = None

print(age)

None


In [86]:
path_to_file = data_folder + 'quotes-2015.json.bz2' 
path_to_out = data_folder + 'quotes-2015-extended.json.bz2'

iter = 0

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            iter += 1
            if any(substring in instance['quotation'] for substring in lexical_field) and instance['qids'] != []: #We keep only quotation containing words of the lexical field and where there is a speaker
                speaker = speakers.loc[speakers['id'] == instance['qids'][0]].squeeze()
                
                if speaker.nationality is not None: 
                    if any(substring == 'Q30' for substring in speaker.nationality): #keep american speaker

                        #We add the gender
                        instance['gender'] = 'male' if (speaker['gender'][0] == 'Q6581097') else 'female' 

                        #We add the nationalities
                        instance['nationality'] = []
                        for i in speaker['nationality']: 
                            nat = label.loc[i]['Label']
                            instance['nationality'].append(nat)

                        #We add the occupations
                        instance['occupation'] = []
                        for i in speaker['occupation']:
                            occ = label.loc[i]['Label']
                            instance['occupation'].append(occ)

                        #We add the date of birth
                        try:
                            born = datetime.strptime(speaker.date_of_birth[0][1:11], "%Y-%m-%d").date()
                            today = date.today()
                            age = today.year - born.year - ((today.month, today.day) < (born.month, born.day))
                        except:
                            age = None
                        instance['age'] = age

                        # instance['ethnic_group'] = label.loc[speaker.ethnic_group]['Label'] if (speaker.ethnic_group is not None) else None

                        if speaker.party is not None:
                            instance['party'] = []
                            for i in speaker['party']:
                                part = label.loc[i]['Label']
                                instance['party'].append(part)
                        else:
                            instance['party'] = None

                        # instance['religion'] = label.loc[speaker.religion]['Label'] if (speaker.religion is not None) else None
                        d_file.write((json.dumps(instance)+'\n').encode('utf-8')) # writing in the new file
  
            #to test
            if iter > 2000:
                break

{'quoteID': '2015-12-08-003771',
 'quotation': 'After the failure of the U.S. Congress to pass the provision last week, North Carolina should take the lead on saying no to terror suspects getting guns,',
 'speaker': 'Roy Cooper',
 'qids': ['Q16106910', 'Q7372694', 'Q7372695'],
 'date': '2015-12-08 16:15:05',
 'numOccurrences': 1,
 'probas': [['Roy Cooper', '0.928'],
  ['None', '0.0574'],
  ['Frank Perry', '0.0146']],
 'urls': ['http://digtriad.com/story/news/2015/12/08/should-people-no-fly-list-allowed-buy-guns/76996690/'],
 'phase': 'E',
 'gender': 'male',
 'nationality': ['United States of America'],
 'occupation': ['politician'],
 'age': 76,
 'party': ['Republican Party']}

{'quoteID': '2015-11-14-003251',
 'quotation': 'And I will tell you what -- you can say what you want, if they had guns, if our people had guns, if they were allowed to carry, it would have been a much, much different situation,',
 'speaker': 'Donald Trump',
 'qids': ['Q22686', 'Q27947481'],
 'date': '2015-11-14 22:45:14',
 'numOccurrences': 6,
 'probas': [['Donald Trump', '0.798'], ['None', '0.202']],
 'urls': ['http://news.yahoo.com/paris-wouldve-different-french-armed-trump-224514131.html',
  'http://www.themalaymailonline.com/world/article/paris-wouldve-been-different-if-french-armed-says-trump',
  'http://rappler.com/world/regions/us-canada/112824-paris-attacks-donald-trump',
  'http://malaysiandigest.com/world/579053-trump-paris-would-ve-been-different-if-french-armed.html',
  'http://www.freemalaysiatoday.com/category/world/2015/11/15/trump-paris-wouldve-been-different-if-french-armed/',
  'http://hotair.com/headlines/archives/2015/11/15/if-they-were-allowed-to-carry-it-would-ha

# Analysis

In [14]:
def process_chunk(chunk):
    print(f'Processing chunk with {len(chunk)} rows')
    display(chunk)
    # print(chunk.quotation[1])
    # chunk['scores'] = chunk['quotation'].apply(lambda review: sid.polarity_scores(review))
    # chunk['compound']  = chunk['scores'].apply(lambda score_dict: score_dict['compound'])
    # for id, row in chunk.iterrows():
    #     print(chunk.quotation[id])
    #     print(chunk.compound[id])

with pd.read_json(data_folder + 'quotes-2015-extended.json.bz2', lines=True, compression='bz2', chunksize=10000) as df_reader:
    for chunk in df_reader:
        process_chunk(chunk)
        break

Processing chunk with 4 rows
... through you -- the vibrant young athletes who are here today -- and through your hosts of modern Germany who have graciously welcomed you here, we have the opportunity to look forward, not backward.
' It is not now, nor has it ever been, the gold standard of trade agreements,' Sanders said. And he reached back to Clinton's 2002 vote to support the war in Iraq, an issue that plagued her eight years ago when she took the stage here. `When I came to that fork in the road I took the right road, even though it was not the popular road at the time,' he said.
-0.4822
... through you -- the vibrant young athletes who are here today -- and through your hosts of modern Germany who have graciously welcomed you here, we have the opportunity to look forward, not backward.
0.8979
I am incredibly pleased that my message is resonating with people all over the United States. There are so many American's that are ready to make our country great again, and I am the only o